In [9]:
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [10]:
data = open('./data/input.txt','r')
data = list(data)
data_use = []
data_f = []
index = 0
temp = ''
for d in data:
    if d == "<start>\r\n":
        index = 1
        continue
    if d == '<end>\r\n':
        index = 0
    if index == 1:
        temp = temp + str(d)
    if index == 0:
        data_use.append(temp)
        temp = ''
#print(d)
for d in data_use:
    if len(d) > 40:
        data_f.append(d)
data_use = data_f
all_character = set()
counter = 0
for d in data_use:
    for dd in d:
        all_character.add(dd)
all_character = list(all_character)
nletter = len(all_character)

In [11]:
print(data_use[2])

XIV?me si?cle)
Z:Transcrit et/ou corrig? par Michel BELLON - 2005-03-27
Z:Pour toute observation mailto:galouvielle@free.fr
M:2/2
L:1/4
Q:1/4=110
K:Fmaj
V:1
dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G|F4|dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G/2A/4F/4|G2 zG|
w:Stel-__la splen-dens in mon-____te ut so-lis ra-_di-um Mi-__ra-cu-lis ser-ra-____to ex-au-di po-_pu-__lum  Prin-
Gd eg|ef/2d/2 ce|fd c/2B/2B/2A/2|G2 zG|Gd eg|ef/2d/2 ce|fd c/2B/2B/2A/2|G2 z2||
w:ci-pes et ma-gna-__ tes ex-stir-pe re-_gi-_a sae-cu-li po-tes-ta-__tes ob-ten-ta ve-_ni-_a
dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G|F4|dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G/2A/4F/4|G4|]
w:Pe-__cca mi num pro-cla-____mant tu den tes pec-_to-ra Po-__pli te fle-xo cla-____mant hic A-ve Ma-_ri-__a
V:2
G2 GB|dG F2|B2 cd|GF cd|f4| G2 GB|dG F2|B2 cd|GF Bc|d2 zd-|
d G2 G|cB A2|FG Bc|d2 zd-|d G2 G|cB A2|FG Bc|d2 z2|
G2 GB|dG F2|B2 cd|GF cd|f4| G2 GB|dG F2|B2 cd|GF Bc|d4|]



In [12]:
def lineToTensor(line):
    tensor = torch.zeros(len(line),nletter)
    position = np.zeros(len(line))
    for i, letter in enumerate(line):
        position[i] = all_character.index(letter)
        tensor[i][all_character.index(letter)] = 1
    position = torch.from_numpy(position[1:]).long()
    return autograd.Variable(tensor[:-1,:]).cuda(),autograd.Variable(position).cuda()

In [13]:
def batch_g(data,batch_size):
    lyric = np.random.randint(len(data))
    #print(len(data[lyric]))
    if len(data[lyric])-batch_size >0:
        interval_s = np.random.randint(len(data[lyric])-batch_size)
        return data[lyric][interval_s:interval_s+batch_size]
    else:
        return []

In [14]:
class LSTM(nn.Module):
    def __init__(self,letter_dim,hidden_dim):
        super(LSTM, self).__init__()
        self.letter_dim = letter_dim
        self.hidden_dim = hidden_dim
        self.hidden = self.init_hidden()
        
        self.lstm = nn.LSTM(letter_dim,hidden_dim)
        self.hidden2out = nn.Linear(hidden_dim,letter_dim)
    def init_hidden(self):
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)).cuda(),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)).cuda())
    def forward(self,line):
        lstm_out, self.hidden = self.lstm(line.view(len(line),1,-1), self.hidden)
        out_space = self.hidden2out(lstm_out.view(len(lstm_out),-1))
#         out_scores = F.softmax(out_space / 1.0,dim = 1)
        return out_space

In [15]:
hidden_dim = 100
batch_size = 30
lstm = LSTM(nletter,hidden_dim).cuda()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm.parameters(), lr=0.001)

In [16]:
for epoch in range(10000):
    lstm.zero_grad()
    batch = batch_g(data_use,batch_size)
#     print(batch)
#     print('\n')
    if len(batch)>0:
        batch_in,batch_t = lineToTensor(batch)
        lstm.hidden = lstm.init_hidden()
        out_scores = lstm(batch_in)
        loss = loss_function(out_scores,batch_t)
        if epoch % 1000 == 0:
            print(loss)
        loss.backward()
        optimizer.step()

Variable containing:
 4.5470
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.9171
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.7177
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.2790
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.6153
[torch.cuda.FloatTensor of size 1 (GPU 0)]



KeyboardInterrupt: 

In [20]:
print(F.softmax(out_scores / 1.0,dim = 1).data.cpu().numpy()[1])

[  1.71104241e-02   1.10607070e-03   1.33918819e-03   3.31502379e-05
   1.41654843e-02   2.16764340e-04   1.50187581e-04   5.82695466e-05
   2.38036358e-04   5.76608318e-06   2.19230401e-06   2.81575958e-05
   3.62753053e-06   1.39874965e-08   5.09926060e-04   3.07068348e-01
   7.75646186e-05   2.20940914e-03   4.11001565e-05   4.69862454e-04
   1.01717458e-04   5.64067566e-04   4.96421808e-06   5.13616018e-04
   3.81924256e-05   2.44627520e-02   2.21263953e-02   1.59723204e-04
   6.57657365e-05   1.58825220e-04   5.27087366e-03   1.58715648e-05
   5.98427562e-07   1.70586172e-05   6.88771161e-06   4.37294686e-04
   2.73574522e-04   1.19618692e-01   2.09524371e-02   1.17771442e-05
   8.54894810e-04   3.89511610e-04   3.04890855e-05   5.65584924e-05
   2.91107081e-05   1.00196747e-03   2.32100865e-06   1.12615282e-07
   6.70639682e-04   2.31218830e-01   1.65626942e-03   3.25664914e-05
   1.29153822e-02   3.10283471e-02   1.64982153e-03   6.98874192e-06
   1.75418268e-06   5.68510814e-06

In [21]:
present = torch.zeros(1,nletter)
present[0][all_character.index('X')] = 1
present = autograd.Variable(present).cuda()
lyric = []
lyric.append('X')
for num_generation in range(400):
    #print(lstm.forward(present).cpu())
    next_char_index = int(torch.distributions.Categorical(F.softmax(lstm(present)/1.0,dim = 1).data.cpu()).sample().numpy())
    next_char = all_character[next_char_index]
    lyric.append(next_char)
    present = torch.zeros(1,nletter)
    present[0][all_character.index(next_char)] = 1
    present = autograd.Variable(present).cuda()

In [22]:
print(str(lyric))

['X', '2', 'G', '2', ' ', 'B', '2', 'G', 'f', '|', 'e', '3', '@', 'd', 'c', 'e', 'd', ' ', '|', ' ', '(', '3', '.', 'g', 'z', 'f', 'g', '.', 'g', '3', ' ', 'd', 'c', 'a', '|', '\r', '\n', 'B', 'd', 'B', ' ', 'B', 'A', 'B', ' ', '|', ' ', 'G', '3', ' ', 'd', 'e', 'f', '|', '\r', '\n', 'G', 'd', 'c', 'A', '|', '\r', '\n', 'E', '2', ' ', 'A', '2', ')', '\r', '\n', 'N', '2', ' ', '|', ' ', '.', '7', '0', '1', ':', 'F', '3', ' ', 'C', ' ', 'D', ',', ' ', 'S', 'a', ' ', 'd', 'e', 'l', ' ', 'e', ',', ' ', 'H', 'b', 'i', 'n', '\r', '\n', 'R', ':', 's', 'o', 'l', ' ', 'e', 'e', '\r', '\n', 'R', ':', 's', 'e', 'e', 'd', 'a', 'n', 'o', ' ', 'b', 'a', 'l', 'e', ' ', 'd', 'o', 'q', 't', 'e', 'r', 'i', 'e', 't', ' ', 'C', 'o', 'r', 'V', 'a', 'g', '?', ':', '\r', '\n', 'S', ':', 'C', 'a', 'n', 'd', 'e', 'c', 'd', ':', ' ', '\\', 'i', 'd', 't', 'r', 'e', '@', 'b', 's', 'e', 'r', 'v', 'e', '\r', '\n', 'R', ':', 'C', 'h', 'a', 'n', ' ', 's', 'h', 'i', 'm', ' ', 't', 'o', 'u', 't', ' ', 'o', 'l', 'i', 'o

In [23]:
Music = ""
for char in lyric:
    Music = Music + char
print(Music)

X2G2 B2Gf|e3@dced | (3.gzfg.g3 dca|
BdB BAB | G3 def|
GdcA|
E2 A2)
N2 | .701:F3 C D, Sa del e, Hbin
R:sol ee
R:seedano bale doqteriet CorVag?:
S:Candecd: \idtre@bserve
R:Chan shim tout olioek Wira(
T:aadlwe
P:Slid toz Woska:
T:+irnellin s0macesc, Pour Michens Bf anye Roucs
R:slide
Z:id:hn-sereg- a-77
M:C:
K:G
ABBA2 :EFE|GBBcd Adcrameldub, The
Z:id:hn-reel-178
M:C|
K:C
c2dB BcA2|
